In [1]:
import datetime as dt
import pandas as pd
import matplotlib as plt

In [2]:
pip install Lifetimes

Note: you may need to restart the kernel to use updated packages.


In [3]:
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from sklearn.preprocessing import MinMaxScaler

---------

# Görev 1: Veriyi Anlama ve Hazırlama

---------

- Invoice: Fatura Numarası ( Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder )
- StockCode: Ürün kodu ( Her bir ürün için eşsiz )
- Description: Ürün ismi
- Quantity: Ürün adedi ( Faturalardaki ürünlerden kaçar tane satıldığı)
- InvoiceDate: Fatura tarihi
- Price: Fatura fiyatı ( Sterlin )
- CustomerID: Eşsiz müşteri numarası
- Country: Ülke ismi

### Adım 1: Online Retail II excelindeki 2010-2011 verisini okuyunuz. Oluşturduğunuz dataframe’in kopyasını oluşturunuz.

In [5]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [6]:
df = df_.copy()

---------

### Adım 2: Veri setinin betimsel istatistiklerini inceleyiniz.

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.00,9.55,218.08,-80995.00,1.00,3.00,10.00,80995.00
Price,541910.00,4.61,96.76,-11062.06,1.25,2.08,4.13,38970.00
Customer ID,406830.00,15287.68,1713.60,12346.00,13953.00,15152.00,16791.00,18287.00


---------

### Adım 3: Veri setinde eksik gözlem var mı? Varsa hangi değişkende kaç tane eksik gözlem vardır?

In [9]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

---------

### Adım 4: Eksik gözlemleri veri setinden çıkartınız. Çıkarma işleminde ‘inplace=True’ parametresini kullanınız.

In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum() # kontrol edelim , null değerlerimiz veri setinden kaldırılmış.

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

---------

### Adım 5: Eşsiz ürün sayısı kaçtır?

In [12]:
df["Description"].nunique()

3896

---------

### Adım 6: Hangi üründen kaçar tane vardır?

In [13]:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
Name: Description, dtype: int64

---------

### Adım 7: En çok sipariş edilen 5 ürünü çoktan aza doğru sıralayınız

In [14]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


---------

### Adım 8: Faturalardaki ‘C’ iptal edilen işlemleri göstermektedir. İptal edilen işlemleri veri setinden çıkartınız.

In [15]:
df = df[~df["Invoice"].str.contains("C", na=False)] # df datasında colon adı Invoice olanın başında c olanları sil

---------

### Adım 9: Fatura başına elde edilen toplam kazancı ifade eden ‘TotalPrice’ adında bir değişken oluşturunuz 

In [16]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [17]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.00,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.00,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.00,France,16.60
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.00,France,14.85


---------

# Görev 2: RFM Metriklerinin Hesaplanması

---------

### Adım 1: Recency, Frequency ve Monetary tanımlarını yapınız.

- Recency(yenilik : analizin yapıldığı tarih- müşterinein son satın aldığı tarih)
- Frequency(müşterinin toplan satın alması) 
- Monetary(toplam parasal değer)

---------

### Adım 2: Müşteri özelinde Recency, Frequency ve Monetary metriklerini groupby, agg ve lambda ile hesaplayınız.
- Adım 3: Hesapladığınız metrikleri rfm isimli bir değişkene atayınız
- Adım 4: Oluşturduğunuz metriklerin isimlerini recency, frequency ve monetary olarak değiştiriniz.
* recency değeri için bugünün tarihini (2011, 12, 11) olarak kabul ediniz.
* rfm dataframe’ini oluşturduktan sonra veri setini "monetary>0" olacak şekilde filtreleyiniz

In [18]:
df["InvoiceDate"].max() #müşterinein son satın aldığı tarih
today_date = dt.datetime(2011, 12, 11) #recency değeri için bugünün tarihini (2011, 12, 11) olarak kabul ediniz.
type(today_date)

rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,## Recency
                                     'Invoice': lambda Invoice: Invoice.nunique(),#Frequency
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})#Monetary


rfm.columns = ['recency', 'frequency', 'monetary'] # kolon isimlerini yeniden tanımladık

rfm = rfm[rfm["monetary"] > 0]

rfm.head()



,recency,frequency,monetary
Customer ID,,,
12346.00,326,1,77183.60
12347.00,3,7,4310.00
12348.00,76,4,1797.24
12349.00,19,1,1757.55
12350.00,311,1,334.40


---------

# Görev 3: RFM Skorlarının Oluşturulması ve Tek bir Değişkene Çevrilmesi

### Adım 1: Recency, Frequency ve Monetary metriklerini qcut yardımı ile 1-5 arasında skorlara çeviriniz.
- Adım 2: Bu skorları recency_score, frequency_score ve monetary_score olarak kaydediniz.
- Adım 3: recency_score ve frequency_score’u tek bir değişken olarak ifade ediniz ve RF_SCORE olarak kaydediniz.

In [19]:
# 1 -2
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]) 
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
# 3
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))


In [20]:
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.00,326,1,77183.60,1,1,5,11
12347.00,3,7,4310.00,5,5,5,55
12348.00,76,4,1797.24,2,4,4,24
12349.00,19,1,1757.55,4,1,4,41
12350.00,311,1,334.40,1,1,2,11
...,...,...,...,...,...,...,...
18280.00,278,1,180.60,1,2,1,12
18281.00,181,1,80.82,1,2,1,12
18282.00,8,2,178.05,5,3,1,53


---------

# Görev 4: RF Skorunun Segment Olarak Tanımlanması

---------

### Adım 1: Oluşturulan RF skorları için segment tanımlamaları yapınız.
### Adım 2: Aşağıdaki seg_map yardımı ile skorları segmentlere çeviriniz

In [21]:
   seg_map = {
    r'[1-2][1-2]': 'hibernating', #1. elemanda 1 yada 2  görürsen , 2.elamanda 1 yada 2 gçrürsen hibernating' yaz demek
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',  # 2. elemanda  görürsen köşeli parantez - varsa yada demek.
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',  # 1 elemanında ve 2. elemanında 3 görürsen bu isimldirmeyi yap
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
   }

In [22]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True) # RFM_SCORE göre segmetleri tanımladık

---------

# Görev 5: Aksiyon Zamanı !

### Adım 1: Önemli gördüğünü 3 segmenti seçiniz. Bu üç segmenti hem aksiyon kararları açısından hemde segmentlerin yapısı açısından(ortalama RFM değerleri) yorumlayınız.

In [23]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean"])

,recency,frequency,monetary
,mean,mean,mean
segment,,,
about_to_sleep,53.31,1.16,471.99
at_Risk,153.79,2.88,1084.54
cant_loose,132.97,8.38,2796.16
champions,6.36,12.42,6857.96
hibernating,217.61,1.10,488.64
loyal_customers,33.61,6.48,2864.25
need_attention,52.43,2.33,897.63
new_customers,7.43,1.00,388.21


- champions katogerisine sahip müşterilerin getirilerinün  yüksek olduğunu görüyoruz bu müşteriler için özel kampanyalar , hediyeler verilebilir.



### 2: "Loyal Customers" sınıfına ait customer ID'leri seçerek excel çıktısını alınız.

In [ ]:
target_customers = rfm[rfm["segment"].isin(["loyal_customers"])]|df["Customer ID"]

In [ ]:
rfm.head()

In [ ]:
target_customers.to_xlsx("hedef_müşteri_id.xlsx", index=False)